In [2]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 48.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=e512f39b06cf931f1d938ab92b7a8fae9ae94c4b12af06ff2f339686e7a2e6bb
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [1]:
import numpy as np
import pandas as pd
from pandas import read_csv
from datetime import datetime
from matplotlib import pyplot
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")
from warnings import catch_warnings
from warnings import filterwarnings
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pyspark.sql.session import SparkSession
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed

spark = SparkSession.builder.getOrCreate()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import time
import os
import pickle
import sklearn

In [3]:
revenue_weekly = spark.read.option("header",True).csv("/content/RevenueSince2017Weekly.csv")

In [4]:
display(revenue_weekly)

DataFrame[Year: string, Week: string, Revenue: string, Volume: string]

In [5]:
lob = 'Total'

In [6]:
revenue_weekly_df = revenue_weekly.toPandas()

In [7]:
revenue_weekly_df['Year'] = revenue_weekly_df['Year'].astype(int)
revenue_weekly_df['Week'] = revenue_weekly_df['Week'].astype(int)
revenue_weekly_df['Revenue'] = revenue_weekly_df['Revenue'].astype(float)
num_weeks = len(revenue_weekly_df)

In [8]:
display(revenue_weekly_df)

,Year,Week,Revenue,Volume
0,2017,1,1494.762,13
1,2017,2,3155.840,12
2,2017,3,14501.530,43
3,2017,4,10780.378,58
4,2017,5,13961.837,68
...,...,...,...,...
274,2022,13,581314.822,3950
275,2022,14,557168.606,3456
276,2022,15,482892.386,2825
277,2022,16,442427.949,2769


In [9]:
def Insert_row_(row_number, df, year, week):
    df1 = df[0:row_number]
    df2 = df[row_number:]
    df1.loc[row_number]=[year, week, 0]
    df_result = pd.concat([df1, df2])
    df_result.index = [*range(df_result.shape[0])]
    return df_result

In [10]:
start_year = revenue_weekly_df.iloc[0,1]
start_week = revenue_weekly_df.iloc[0,2]
count = 1
for i in range(1,num_weeks):

    if count == 52:
        start_year = start_year + 1
        start_week = 0
        count = 0
    if ((start_year == revenue_weekly_df.iloc[i,0]) and ((start_week + 1) != revenue_weekly_df.iloc[i,1])):
        revenue_weekly_df = Insert_row_(i, revenue_weekly_df, start_year, start_week + 1)
    start_week = start_week + 1
    count = count + 1

In [11]:
display(revenue_weekly_df)


,Year,Week,Revenue,Volume
0,2017,1,1494.762,13
1,2017,2,3155.840,12
2,2017,3,14501.530,43
3,2017,4,10780.378,58
4,2017,5,13961.837,68
...,...,...,...,...
274,2022,13,581314.822,3950
275,2022,14,557168.606,3456
276,2022,15,482892.386,2825
277,2022,16,442427.949,2769


In [12]:
print(revenue_weekly_df.dtypes)

Year         int64
Week         int64
Revenue    float64
Volume      object
dtype: object


In [13]:
X = revenue_weekly_df['Revenue'].values

In [14]:
for i in range(len(revenue_weekly_df)):
    if revenue_weekly_df.iloc[i,0] == 2022 and revenue_weekly_df.iloc[i,1] == 1:
        size = i
print(size)

262


In [15]:
print(size)
print(num_weeks)

262
279


List to store Evaluation scores corresponding to every config

In [16]:
Evaluation_CFG = []

Creating Methods of Grid Search Framwework

In [17]:
# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

In [18]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[0:size] , data[size:num_weeks+1]

In [19]:
def AddEntrytoList(cfg, rmse, mae, r_squared):
  temp_list = []
  for param in cfg:
    
    if type(param) != str:
      for sub_param in param:
        temp_list.append(sub_param)
    
    else:
      temp_list.append(sub_param)

  temp_list.append(rmse)
  temp_list.append(mae)
  temp_list.append(r_squared)
  Evaluation_CFG.append(temp_list)

  #print(Evaluation_CFG)

In [20]:
# walk-forward validation for univariate data
def evaluate_SARIMA(data, n_test, cfg):
  predictions = list()
  # split dataset
  train, test = train_test_split(data, n_test)
  # seed history with training dataset
  history = [x for x in train]
  # step over each time-step in the test set
  for i in range(len(test)):
    # fit model and make forecast for history
    yhat = sarima_forecast(history, cfg)
    # store forecast in list of predictions
    predictions.append(yhat)
    # add actual observation to history for the next loop
    history.append(test[i])

  #Calculate evaluation metrics
  rmse = sqrt(mean_squared_error(test, predictions))
  # print('Test RMSE: %d' % rmse)
  r_squared = r2_score(test, predictions)
  # print('Test R-squared: %.2f' % r_squared)
  mae = mean_absolute_error(test, predictions)
  # print('Test MAE: %d' % mae)

  AddEntrytoList(cfg, rmse, mae, r_squared)
  #print(cfg)
                 
  return r_squared

In [21]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = evaluate_SARIMA(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = evaluate_SARIMA(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
 
	return (key, result)

In [22]:
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):

	scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1], reverse=True)
	return scores

In [23]:
# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0]
	D_params = [0]
	Q_params = [0]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

In [24]:
# model configs
cfg_list = sarima_configs()

In [25]:
# grid search
scores = grid_search(X, cfg_list, num_weeks)
print('done')

 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] -15.875
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] -6.678
 > Model[[(0, 0, 0), (0, 0, 0, 0), 't']] -1.561
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'ct']] -1.802
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'n']] -5.075
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'c']] -2.224
 > Model[[(0, 0, 1), (0, 0, 0, 0), 't']] -0.285
 > Model[[(0, 0, 1), (0, 0, 0, 0), 'ct']] -0.428
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'n']] -2.873
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'c']] -0.956
 > Model[[(0, 0, 2), (0, 0, 0, 0), 't']] 0.136
 > Model[[(0, 0, 2), (0, 0, 0, 0), 'ct']] 0.034
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'n']] 0.363
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'c']] 0.363
 > Model[[(0, 1, 0), (0, 0, 0, 0), 't']] 0.357
 > Model[[(0, 1, 0), (0, 0, 0, 0), 'ct']] 0.351
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'n']] 0.323
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'c']] 0.329
 > Model[[(0, 1, 1), (0, 0, 0, 0), 't']] 0.322
 > Model[[(0, 1, 1), (0, 0, 0, 0), 'ct']] 0.304
 > Model[[(0, 1, 2), (0, 0, 0, 0), 'n']] 0.3

In [26]:
Evaluation_CFG

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  472220.89745837054,
  458015.5781764706,
  -15.875087769673673],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  318536.77453516325,
  298186.5049193381,
  -6.678471773325615],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  183951.34635134626,
  164128.4258737178,
  -1.5607169574830677],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  192417.3722413866,
  173515.15432708213,
  -1.8018455707503747],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  283340.93180557713,
  260557.0764591838,
  -5.075391531058687],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  206398.04362997858,
  186536.9630984408,
  -2.223790333737995],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  130309.7772747634,
  103396.35524902756,
  -0.28501927140749195],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  137387.09936685555,
  111456.2857629942,
  -0.42839244578268243],
 [0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  226228.002989832,
  203979.56651124055,
  -2.8730076917254035],
 [0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,

In [27]:
# Create the pandas DataFrame
df = pd.DataFrame(Evaluation_CFG, columns = ['p', 'd' ,'q', 'P', 'D', 'Q', 'm', 't', 'rmse', 'mae', 'r_squared'])

In [28]:
df

,p,d,q,P,D,Q,m,t,rmse,mae,r_squared
0,0,0,0,0,0,0,0,0,472220.897458,458015.578176,-15.875088
1,0,0,0,0,0,0,0,0,318536.774535,298186.504919,-6.678472
2,0,0,0,0,0,0,0,0,183951.346351,164128.425874,-1.560717
3,0,0,0,0,0,0,0,0,192417.372241,173515.154327,-1.801846
4,0,0,1,0,0,0,0,0,283340.931806,260557.076459,-5.075392
...,...,...,...,...,...,...,...,...,...,...,...
67,2,1,1,0,0,0,0,0,97219.967175,81743.882909,0.284736
68,2,1,2,0,0,0,0,0,101760.234362,83456.516454,0.216369
69,2,1,2,0,0,0,0,0,101138.077990,83911.661410,0.225921
70,2,1,2,0,0,0,0,0,100486.567981,82379.216398,0.235862


In [29]:
#Sort in descending order
df_sorted = df.sort_values(by = ['r_squared'])

In [30]:
df_sorted

,p,d,q,P,D,Q,m,t,rmse,mae,r_squared
0,0,0,0,0,0,0,0,0,472220.897458,458015.578176,-15.875088
1,0,0,0,0,0,0,0,0,318536.774535,298186.504919,-6.678472
4,0,0,1,0,0,0,0,0,283340.931806,260557.076459,-5.075392
8,0,0,2,0,0,0,0,0,226228.002990,203979.566511,-2.873008
5,0,0,1,0,0,0,0,0,206398.043630,186536.963098,-2.223790
...,...,...,...,...,...,...,...,...,...,...,...
50,2,0,0,0,0,0,0,0,89012.406132,71160.382671,0.400407
54,2,0,1,0,0,0,0,0,88945.569830,71381.597047,0.401307
30,1,0,1,0,0,0,0,0,88410.187354,69980.675467,0.408492
34,1,0,2,0,0,0,0,0,87641.689082,69198.432784,0.418731
